In [2]:
pip install tqdm

     |████████████████████████████████| 78 kB 1.9 MB/s             
Note: you may need to restart the kernel to use updated packages.


In [3]:
import numpy as np
import pandas as pd

import requests
import json
import time
#from bs4 import BeautifulSoup
#from lxml import etree
from pandas import DataFrame
from tqdm import tqdm

import warnings
warnings.filterwarnings("ignore")

In [5]:
cities_1k_df = pd.read_csv(r'us-cities-top-1k.csv')
cities_1k_df['cities_states'] = cities_1k_df['City']+','+cities_1k_df['State']
cities_df = cities_1k_df.copy().sort_values(by=['Population'])

cities_df

,City,State,Population,lat,lon,cities_states
961,Panama City,Florida,36877,30.158813,-85.660206,"Panama City,Florida"
552,Beloit,Wisconsin,36888,42.508348,-89.031777,"Beloit,Wisconsin"
434,Spanish Fork,Utah,36956,40.114955,-111.654923,"Spanish Fork,Utah"
722,Keizer,Oregon,37064,44.990119,-123.026208,"Keizer,Oregon"
346,Weslaco,Texas,37093,26.159519,-97.990837,"Weslaco,Texas"
...,...,...,...,...,...,...
53,Philadelphia,Pennsylvania,1553165,39.952584,-75.165222,"Philadelphia,Pennsylvania"
646,Houston,Texas,2195914,29.760427,-95.369803,"Houston,Texas"
594,Chicago,Illinois,2718782,41.878114,-87.629798,"Chicago,Illinois"
953,Los Angeles,California,3884307,34.052234,-118.243685,"Los Angeles,California"


# 爬数据

In [12]:
#Define Search
search_location = cities_df['cities_states'].drop_duplicates(keep = 'first').tolist()
search_location.remove('Lima,Ohio')
search_categories = 'restaurants, All'
#search_location = cities_df['State'].drop_duplicates().tolist()
search_list = ['restaurant, curbside pickup']
search_limit = 50
offset_limit = 50

In [13]:
len(search_location)

999

In [18]:
#定义搜索公式
def search_restaurant(set_location,offset_number,search_term):
    # 根据输入的地点和循环次数抓取该地点的所有餐厅数据

    url = 'https://api.yelp.com/v3/businesses/search' #Yelp Fusion
    headers = {'Authorization': 'Bearer {}'.format('6y1SsdLqGVrvbcgVHJb6AKJ0aGJzDJwUnT2F1FMpKElB15TUvbagp9gQLPqEJjyq69EPeQuRsatOENV5y3-qMGBw-yE6-ayOEQVY90Bu84RdaZ2id0adh27tzbZnYnYx')} #个人密钥
    url_params = { #parameters passed to the API
        'term': search_term,
        'categories': search_categories,
        'location': set_location,
        'offset': offset_number,
        'limit': search_limit # Maximum return of results per request (ref: API documentation).
    }

    restaurant_response = requests.get(url, headers=headers, params=url_params).json()
    restaurant_output = pd.DataFrame.from_dict(restaurant_response['businesses'])
    return restaurant_output # Returns a df

In [21]:
#定义城市循环公式
def loop_city(start_city_number,target_city_number, offset_number, search_term):
    loop_city_merge = pd.DataFrame()
    for i in np.arange(start_city_number,target_city_number,1) :
        loop_city_append = search_restaurant(search_location[i], offset_number, search_term)
        loop_city_merge = DataFrame(loop_city_merge.append(loop_city_append, ignore_index=True))
    return loop_city_merge

In [32]:
#定义同城循环公式
def loop_restaurant(target_city_number, search_term):
    loop_restaurant_merge = pd.DataFrame()
    for n in np.arange(0, offset_limit, 50):
        loop_restaurant_append = loop_city(target_city_number,n, search_term)
        loop_restaurant_merge = DataFrame(loop_restaurant_merge.append(loop_restaurant_append, ignore_index=True))
    return loop_restaurant_merge

In [33]:
#定义种类循环公式
def loop_search(target_city_number):
    loop_search_merge = pd.DataFrame()
    for m in np.arange(0, len(search_list), 1):
        loop_search_append = loop_restaurant(target_city_number,search_list[m])
        loop_search_merge = DataFrame(loop_search_merge.append(loop_search_append, ignore_index=True))
    return loop_search_merge

In [32]:
data_5 = loop_city(0,999,200,'curbside pickup')

data_5

,id,alias,name,image_url,is_closed,url,review_count,categories,rating,coordinates,transactions,price,location,phone,display_phone,distance
0,dTkYItaR86zUQaOZ78_b_A,subway-portland-183,Subway,https://s3-media4.fl.yelpcdn.com/bphoto/I91rjK...,False,https://www.yelp.com/biz/subway-portland-183?a...,55,"[{'alias': 'sandwiches', 'title': 'Sandwiches'}]",1.5,"{'latitude': 45.5353041, 'longitude': -122.652...","[pickup, delivery]",$,"{'address1': '1301 NE Broadway', 'address2': '...",+15032844782,(503) 284-4782,14108.016517
1,WS1u2QJR3T32Ca1xAzaYwQ,betty-fayes-cafe-corona-2,Betty Faye's Cafe,https://s3-media3.fl.yelpcdn.com/bphoto/w__0Ia...,False,https://www.yelp.com/biz/betty-fayes-cafe-coro...,360,"[{'alias': 'newamerican', 'title': 'American (...",4.0,"{'latitude': 33.891214, 'longitude': -117.56438}",[delivery],$$,"{'address1': '718 N Main St', 'address2': 'Ste...",+19517343994,(951) 734-3994,28225.646102
2,wk8BKq66cKDkdVNvjaEzYA,panera-bread-chino-hills-4,Panera Bread,https://s3-media4.fl.yelpcdn.com/bphoto/fctaHm...,False,https://www.yelp.com/biz/panera-bread-chino-hi...,403,"[{'alias': 'sandwiches', 'title': 'Sandwiches'...",3.0,"{'latitude': 34.00109999221415, 'longitude': -...","[pickup, delivery]",$$,"{'address1': '13865 City Center Drive', 'addre...",+19095917290,(909) 591-7290,13674.814048
3,z7PbKdqdjZeaKmTMQnyfXg,francoli-gourmet-orange,Francoli Gourmet,https://s3-media4.fl.yelpcdn.com/bphoto/2fMPVh...,False,https://www.yelp.com/biz/francoli-gourmet-oran...,728,"[{'alias': 'italian', 'title': 'Italian'}]",4.0,"{'latitude': 33.7874530255795, 'longitude': -1...","[pickup, delivery]",$$,"{'address1': '100 S Glassell St', 'address2': ...",+17142881077,(714) 288-1077,39830.773751
4,AW1kjhCH4Sp1VZdIDIgrLQ,islands-restaurant-brea,Islands Restaurant,https://s3-media2.fl.yelpcdn.com/bphoto/-ppYvw...,False,https://www.yelp.com/biz/islands-restaurant-br...,386,"[{'alias': 'burgers', 'title': 'Burgers'}, {'a...",3.5,"{'latitude': 33.911669859238, 'longitude': -11...",[delivery],$$,"{'address1': '250 S State College Blvd', 'addr...",+17142561666,(714) 256-1666,29145.506264
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
9356,JXyEaw_p8Kfpqrnjo8TPsA,bareburger-ridgewood-ridgewood,Bareburger - Ridgewood,https://s3-media2.fl.yelpcdn.com/bphoto/A4JeaQ...,False,https://www.yelp.com/biz/bareburger-ridgewood-...,251,"[{'alias': 'burgers', 'title': 'Burgers'}, {'a...",3.5,"{'latitude': 40.98002, 'longitude': -74.1197}","[pickup, delivery]",$$,"{'address1': '15 E Ridgewood Ave', 'address2':...",+12013454207,(201) 345-4207,32296.873325
9357,utY8CzA6P5brtUeImic6eg,nonna-maria-brick-oven-pizza-north-bergen,Nonna Maria Brick Oven Pizza,https://s3-media4.fl.yelpcdn.com/bphoto/5MQtpi...,False,https://www.yelp.com/biz/nonna-maria-brick-ove...,7,"[{'alias': 'pizza', 'title': 'Pizza'}, {'alias...",4.0,"{'latitude': 40.78235, 'longitude': -74.03495}","[pickup, delivery]",NaN,"{'address1': '4108 Tonnele Ave', 'address2': '...",+12013301671,(201) 330-1671,9208.874139
9358,YHLwbFPKv0RU8LBlYcLiJg,applebees-grill-bar-baldwin,Applebee's Grill + Bar,https://s3-media2.fl.yelpcdn.com/bphoto/0D1jqQ...,False,https://www.yelp.com/biz/applebees-grill-bar-b...,95,"[{'alias': 'tradamerican', 'title': 'American ...",3.0,"{'latitude': 40.654955, 'longitude': -73.611555}","[pickup, delivery]",$$,"{'address1': '684 Sunrise Hwy', 'address2': ''...",+15164420580,(516) 442-0580,32759.280602
9359,irUZYVvx7raUuIwjGO8UPw,red-robin-gourmet-burgers-and-brews-clifton,Red Robin Gourmet Burgers and Brews,https://s3-media4.fl.yelpcdn.com/bphoto/iV83Rr...,False,https://www.yelp.com/biz/red-robin-gourmet-bur...,256,"[{'alias': 'burgers', 'title': 'Burgers'}, {'a...",3.0,"{'latitude': 40.827260927976255, 'longitude': ...","[pickup, delivery]",$$,"{'address1': '265 NJ 3 E', 'address2': '', 'ad...",+19734709222,(973) 470-9222,17977.543735


In [33]:
data_5 = data_5.drop_duplicates('id',keep = 'first')

data_5

,id,alias,name,image_url,is_closed,url,review_count,categories,rating,coordinates,transactions,price,location,phone,display_phone,distance
0,dTkYItaR86zUQaOZ78_b_A,subway-portland-183,Subway,https://s3-media4.fl.yelpcdn.com/bphoto/I91rjK...,False,https://www.yelp.com/biz/subway-portland-183?a...,55,"[{'alias': 'sandwiches', 'title': 'Sandwiches'}]",1.5,"{'latitude': 45.5353041, 'longitude': -122.652...","[pickup, delivery]",$,"{'address1': '1301 NE Broadway', 'address2': '...",+15032844782,(503) 284-4782,14108.016517
1,WS1u2QJR3T32Ca1xAzaYwQ,betty-fayes-cafe-corona-2,Betty Faye's Cafe,https://s3-media3.fl.yelpcdn.com/bphoto/w__0Ia...,False,https://www.yelp.com/biz/betty-fayes-cafe-coro...,360,"[{'alias': 'newamerican', 'title': 'American (...",4.0,"{'latitude': 33.891214, 'longitude': -117.56438}",[delivery],$$,"{'address1': '718 N Main St', 'address2': 'Ste...",+19517343994,(951) 734-3994,28225.646102
2,wk8BKq66cKDkdVNvjaEzYA,panera-bread-chino-hills-4,Panera Bread,https://s3-media4.fl.yelpcdn.com/bphoto/fctaHm...,False,https://www.yelp.com/biz/panera-bread-chino-hi...,403,"[{'alias': 'sandwiches', 'title': 'Sandwiches'...",3.0,"{'latitude': 34.00109999221415, 'longitude': -...","[pickup, delivery]",$$,"{'address1': '13865 City Center Drive', 'addre...",+19095917290,(909) 591-7290,13674.814048
3,z7PbKdqdjZeaKmTMQnyfXg,francoli-gourmet-orange,Francoli Gourmet,https://s3-media4.fl.yelpcdn.com/bphoto/2fMPVh...,False,https://www.yelp.com/biz/francoli-gourmet-oran...,728,"[{'alias': 'italian', 'title': 'Italian'}]",4.0,"{'latitude': 33.7874530255795, 'longitude': -1...","[pickup, delivery]",$$,"{'address1': '100 S Glassell St', 'address2': ...",+17142881077,(714) 288-1077,39830.773751
4,AW1kjhCH4Sp1VZdIDIgrLQ,islands-restaurant-brea,Islands Restaurant,https://s3-media2.fl.yelpcdn.com/bphoto/-ppYvw...,False,https://www.yelp.com/biz/islands-restaurant-br...,386,"[{'alias': 'burgers', 'title': 'Burgers'}, {'a...",3.5,"{'latitude': 33.911669859238, 'longitude': -11...",[delivery],$$,"{'address1': '250 S State College Blvd', 'addr...",+17142561666,(714) 256-1666,29145.506264
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
9356,JXyEaw_p8Kfpqrnjo8TPsA,bareburger-ridgewood-ridgewood,Bareburger - Ridgewood,https://s3-media2.fl.yelpcdn.com/bphoto/A4JeaQ...,False,https://www.yelp.com/biz/bareburger-ridgewood-...,251,"[{'alias': 'burgers', 'title': 'Burgers'}, {'a...",3.5,"{'latitude': 40.98002, 'longitude': -74.1197}","[pickup, delivery]",$$,"{'address1': '15 E Ridgewood Ave', 'address2':...",+12013454207,(201) 345-4207,32296.873325
9357,utY8CzA6P5brtUeImic6eg,nonna-maria-brick-oven-pizza-north-bergen,Nonna Maria Brick Oven Pizza,https://s3-media4.fl.yelpcdn.com/bphoto/5MQtpi...,False,https://www.yelp.com/biz/nonna-maria-brick-ove...,7,"[{'alias': 'pizza', 'title': 'Pizza'}, {'alias...",4.0,"{'latitude': 40.78235, 'longitude': -74.03495}","[pickup, delivery]",NaN,"{'address1': '4108 Tonnele Ave', 'address2': '...",+12013301671,(201) 330-1671,9208.874139
9358,YHLwbFPKv0RU8LBlYcLiJg,applebees-grill-bar-baldwin,Applebee's Grill + Bar,https://s3-media2.fl.yelpcdn.com/bphoto/0D1jqQ...,False,https://www.yelp.com/biz/applebees-grill-bar-b...,95,"[{'alias': 'tradamerican', 'title': 'American ...",3.0,"{'latitude': 40.654955, 'longitude': -73.611555}","[pickup, delivery]",$$,"{'address1': '684 Sunrise Hwy', 'address2': ''...",+15164420580,(516) 442-0580,32759.280602
9359,irUZYVvx7raUuIwjGO8UPw,red-robin-gourmet-burgers-and-brews-clifton,Red Robin Gourmet Burgers and Brews,https://s3-media4.fl.yelpcdn.com/bphoto/iV83Rr...,False,https://www.yelp.com/biz/red-robin-gourmet-bur...,256,"[{'alias': 'burgers', 'title': 'Burgers'}, {'a...",3.0,"{'latitude': 40.827260927976255, 'longitude': ...","[pickup, delivery]",$$,"{'address1': '265 NJ 3 E', 'address2': '', 'ad...",+19734709222,(973) 470-9222,17977.543735


In [34]:
data_5.to_csv('city_curbside_citi_5_test.csv')

# 确认是否有户外就餐

In [ ]:
url_list = data['url'].to_list()

In [ ]:
#定义attribute关键词确认公式
def search_detail(url,detail_key):
    yelp_r = requests.get(url)
    yelp_soup = BeautifulSoup(yelp_r.content, 'html.parser')
    yelp_text = str(yelp_soup)
    if detail_key in yelp_text:
        return 1
    else:
        return 0

In [ ]:
#定义循环搜索公式
def confirmation_result(detail_key):
    temp = 0
    total = len(url_list)
    confirmation_list = []
    for link in np.arange(0, len(url_list), 1):
        confirmation = search_detail(url_list[link],detail_key)
        confirmation_list.append(confirmation)
        temp += 1
        print('\r' + '[Progress]:[%s%s]%.2f%%;' % ('█' * int(temp*20/total), ' ' * (20-int(temp*20/total)),float(temp/total*100)), end='')
        time.sleep(0.01)
    return confirmation_list

In [ ]:
No_Outdoor_Seating= confirmation_result('No Outdoor Seating')

In [ ]:
data['No Outdoor Seating'] = No_Outdoor_Seating
data